In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# # !pip install aiosmtpd
# # !uv pip install aiosmtpd

In [ ]:
# isort: off
# stdlib
import os

import sys

# syft absolute
import syft as sy
from syft.util.util import (
    find_base_dir_with_tox_ini,
    get_caller_file_path,
    is_interpreter_jupyter,
)


def add_helper_path_to_python_path() -> None:
    current_path = "."

    # jupyter uses "." which resolves to the notebook
    if not is_interpreter_jupyter():
        # python uses the file which has from syft import test_settings in it
        import_path = get_caller_file_path()
        if import_path:
            current_path = import_path

    base_dir = find_base_dir_with_tox_ini(current_path)
    notebook_helper_path = os.path.join(
        base_dir, "notebooks/scenarios/bigquery/upgradability/0.9.1_helpers"
    )
    sys.path.append(notebook_helper_path)


add_helper_path_to_python_path()
# third party
from email_helpers import SENDER
from email_helpers import create_user
from email_helpers import get_email_server
from email_helpers import make_user
from email_helpers import save_users
# isort: on

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

# Launch server & login

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high-migrations",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
email_server, smtp_server = get_email_server()

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
high_client.users

In [ ]:
# TODO: this should show SyftSuccess?
high_client.api.services.settings.enable_notifications(
    email_sender=SENDER,
    email_server="localhost",
    email_port="9025",
)

In [ ]:
# # TODO: this should show SyftSuccess?
# high_client.api.services.settings.disable_notifications()

# Register users

In [ ]:
num_users = int(os.environ.get("NUM_TEST_USERS", 5))
print(f"registering {num_users} users")

In [ ]:
users = []
email_disable_index = 0
reset_password_index = 1
for i in range(num_users):
    user = make_user()
    user._email_server = email_server
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    if reset_password_index == i:
        user.reset_password = True
    users.append(user)

In [ ]:
save_users(users)

In [ ]:
# stdlib
import asyncio

await asyncio.sleep(5)

## Verify Emails are sent

In [ ]:
# everyone gets a welcome email
server_name = high_client.name
for user in users:
    emails = user.emails
    assert len(emails) == 1
    welcome_email = user.emails[0]
    assert welcome_email.email_from == SENDER
    assert len(welcome_email.email_to) == 1
    assert welcome_email.email_to[0] == user.email
    assert f"Welcome to {server_name}" in welcome_email.email_content

In [ ]:
# one user disables notifications
# one user requests a password reset
no_email_user = None
reset_password_user = None
for user in users:
    user.client = high_client  # get user client
    if user.email_disabled:
        no_email_user = user
        # disable for this user only
        user.client.api.notifications.deactivate()

    if user.reset_password:
        # ask admin for forgot password flow
        user.client.guest().forgot_password(email=user.email)
        assert "Password Reset Requested" in user.emails[1].email_content
        reset_password_user = user

In [ ]:
ds0 = users[0]

In [ ]:
ds0_user = ds0.client.account
ds0_user

In [ ]:
# with sy.raises(
# sy.SyftException(public_message="*tried to update user*"
# ), show=True): this is different on k8s no idea why
with sy.raises(sy.SyftException, show=True):
    ds0.client.users.update(uid=ds0_user.id, role="admin")

In [ ]:
# with sy.raises(sy.SyftException(public_message="*tried to update user*"), show=True):
with sy.raises(sy.SyftException, show=True):
    ds0_user.update(role="admin")

In [ ]:
# TODO: test disabling and re-enabling all notifications

In [ ]:
# high_client.api.services.settings.disable_notifications()
# high_client.api.services.settings.enable_notifications()

## Test reset password

In [ ]:
# # This is necessary as it sets the new token value in user.reset_token
token = reset_password_user.get_token()
token

In [ ]:
# This is necessary as it sets the new password value in user.new_password
passw = reset_password_user.make_new_password()
passw

In [ ]:
assert token
assert passw

In [ ]:
output = reset_password_user.client.guest().reset_password(
    token=reset_password_user.reset_token, new_password=reset_password_user.new_password
)
output

In [ ]:
assert isinstance(output, sy.SyftSuccess)

In [ ]:
# relogin
reset_password_user.relogin()
# reset_password_user.client = reset_password_user.client

## Reset password second time

In [ ]:
reset_password_user.client.guest().forgot_password(email=reset_password_user.email)

In [ ]:
output = reset_password_user.client.guest().reset_password(
    token=reset_password_user.get_token(),
    new_password=reset_password_user.make_new_password(),
)
output

In [ ]:
assert isinstance(output, sy.SyftSuccess)

In [ ]:
# print(f"token:\t\t {reset_password_user.reset_token}\n\
# password:\t {reset_password_user.password}\n\
# new password:\t {reset_password_user.new_password}")

In [ ]:
reset_password_user.update_password()

In [ ]:
# print(f"token:\t\t {reset_password_user.reset_token}\n\
# password:\t {reset_password_user.password}\n\
# new password:\t {reset_password_user.new_password}")

In [ ]:
# relogin
reset_password_user.relogin()

In [ ]:
save_users(users)

In [ ]:
reset_password_user

## Reduce token expiration and try resetting

In [ ]:
# Variable is poorly named, token expiration time is in seconds and not minutes
high_client.api.services.settings.update(pwd_token_config={"token_exp_min": 3})
high_client.refresh()

In [ ]:
# reset_password_user.client.guest().forgot_password(email=reset_password_user.email)

In [ ]:
# Wait 3 seconds to ensure token expires
await asyncio.sleep(3)

In [ ]:
# This should throw a SyftError because we waited too long
# output = reset_password_user.client.guest().reset_password(
#     token=reset_password_user.get_token(),
#     new_password=reset_password_user.make_new_password(),
# )
# output

In [ ]:
# assert isinstance(output, sy.SyftError)

In [ ]:
# # relogin
# with sy.raises(sy.SyftException, show=True):
#     reset_password_user.relogin()

In [ ]:
# Set things back to the the default settings
high_client.api.services.settings.update(pwd_token_config={"token_exp_min": 1800})
high_client.refresh()

# Cleanup

In [ ]:
smtp_server.stop()

In [ ]:
server.land()